In [0]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from tqdm import tqdm, tqdm_notebook
import time
import os
from datetime import datetime
import random
from threading import Thread

In [0]:
#naver_map_seoul_salon_output_data_2020-04-19_final.csv 파일 경로
path = './naver_place_count_total_2020-05-01_final.csv'
#이미지 파일 저장할 디렉토리 설정. 반드시 끝에 /를 포함하셔야 합니다.
# imgDir = 'd:/marytalk/image/reviews/'
#크롤링할 업소 인덱스 번호 설정
#자신의 크롬 버젼에 따라서 Chrome/80.0.3987.163 부분을 수정해주세요
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36", 
            "Accept":"text/html,application/xhtml+xml,application/xml;\
            q=0.9,imgwebp,*/*;q=0.8"}

#업소 CSV파일 불러오기
salonDF = pd.read_csv(path)
salonDF = salonDF[['store_id_only_num', 'store_name']]

#영수증 리뷰 함수            
def receipt_crawler(start, end):
    review_list = []
    rejected_salon_list = []
    empty_list = []
    baseurl = 'https://store.naver.com/hairshops/detail?id='
    for code_index in tqdm_notebook(range(start, end)):
        page_number=0
        time.sleep(random.uniform(2,8)+np.random.rand())
        while 1 :
            receipt_url = baseurl + str(salonDF.store_id_only_num[code_index])+'&tab=receiptReview&tabPage='+str(page_number)
            print(receipt_url)
            html = requests.get(receipt_url, headers=headers)
            time.sleep(6+np.random.rand())
            naver_place = BeautifulSoup(html.text, 'html.parser')

            
            if html.status_code != 200 :
                reject_salon = {"store_id_only_num":salonDF.store_id_only_num[code_index] ,"store_name":salonDF.store_name[code_index]}
                rejected_salon_list.append(reject_salon)
                print(str(code_index) + " "+salonDF.store_name[code_index]+" 리뷰 데이터 불러오기 실패")
                break
                                
            elif naver_place.find('ul', class_="list_receipt_review") is None :
                empty_salon = {"store_id_only_num":salonDF.store_id_only_num[code_index] ,"store_name":salonDF.store_name[code_index]}
                empty_list.append(empty_salon)
                print(str(code_index) + " "+salonDF.store_name[code_index]+" 리뷰 데이터 없음")
                break
            
                
            else : 
                try :
                    lastpage = naver_place.find('span', class_='total').get_text()
                except AttributeError :
                    lastpage = 1
                reviews = naver_place.find('ul', class_="list_receipt_review").find_all('li', class_=["list_item","list_item no_img"])
                for rev in tqdm(reviews) :
                    #하나의 리뷰에서 별점 가져오기 
                    score = rev.find('span', class_='score').get_text()

                    #하나의 리뷰에서 리뷰 가져오기
                    review = rev.find('div', class_='review_txt').get_text()

                    #하나의 리뷰에서 작성자 가져오기
                    reviewer_nickname = rev.find('a', class_='item').get_text()

                    #하나의 리뷰에서 작성일자 가져오기
                    write_date = rev.find_all('span', class_='item')[0].get_text()
                    write_date = datetime.strptime(write_date,'%Y.%m.%d').strftime('%Y%m%d')
                    
                    #하나의 리뷰에서 방문 빈도수 가져오기
                    frequency = rev.find_all('span', class_='item')[1].get_text().rstrip('번째 방문')

                    #pk
                    #pk = salonDF.store_id_only_num[code_index]+'R'+str(write_date)+reviewer_nickname+str(frequency)
                    #pk = "".join(i for i in pk if i not in "\/:*?<>|")
                    rev_info = {'store_id_only_num':salonDF.store_id_only_num[code_index],'store_name': salonDF.store_name[code_index],'score':score, 'review':review, 
                                'reviewer_nickname':reviewer_nickname, 'write_date':write_date, 'frequency':frequency}
                    review_list.append(rev_info)
                    print(str(code_index) + " "+salonDF.store_name[code_index] + " 리뷰 저장.")

                print(str(code_index) + " "+salonDF.store_name[code_index] + " " + str(page_number+1) + "/"+ str(lastpage) + " 페이지 완료")
                if str(page_number+1)==str(lastpage) :
                    print(str(code_index) + " "+salonDF.store_name[code_index] + " 리뷰 저장 완료")
                    html.close()
                    break
                else :
                    page_number+=1
                    time.sleep(5)
                    
    receipt_reviewDF=DataFrame(review_list)
    receipt_reviewDF.to_csv("naver_place_receipt_review(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')
    reject_receipt_salon= DataFrame(rejected_salon_list)
    reject_receipt_salon.to_csv("naver_reject_receipt_salon(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')
    empty_receipt_reviewDF=DataFrame(empty_list)
    empty_receipt_reviewDF.to_csv("naver_empty_receipt_review(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')


def threading_craw(start, end, n_thread) :
    step = np.linspace(start, end, n_thread+1, dtype=int)
    ths = []
    
    for i in tqdm_notebook(range(0,n_thread)) :
        th = Thread(target=receipt_crawler, args=(step[i], step[i+1]))
        ths.append(th)
        th.start()
    
    for th in ths :
        th.join()

In [0]:
#0~2727
start=0
end=1000
n_thread = 15

threading_craw(start, end, n_thread)